In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
# config
data_root = '/hdd1/lim_data/tld_db/test/images'  # 이미지가 저장된 루트 경로
prediction_root = '/path/to/predictions'  # 예측 txt 파일이 저장된 경로
class_names = { 0: "veh_go", 1: "veh_goLeft", 2: "veh_noSign", 3: "veh_stop", 4: "veh_stopLeft", 
               5: "vef_stopWarning", 6: "veh_warning", 7: "ped_go", 8: "ped_noSign", 9: "ped_stop", 
               10: "bus_go", 11: "bus_noSign", 12: "bus_stop", 13: "bus_warning"
               }  # 클래스 이름 사전

In [ ]:
# function 
# YOLO 형식의 txt 파일 읽기 함수
def read_prediction_file(file_path):
    predictions = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            elements = line.strip().split()
            class_id = int(elements[0])
            x_center, y_center, width, height, score = map(float, elements[1:])
            predictions.append((class_id, x_center, y_center, width, height, score))
    return predictions

# 예측 데이터 시각화 함수
def visualize_predictions(image_path, predictions, class_filter=None, score_min=0.0, score_max=1.0):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    
    for class_id, x_center, y_center, width, height, score in predictions:
        if class_filter is not None and class_id != class_filter:
            continue
        if not (score_min <= score < score_max):
            continue

        # YOLO 형식을 이미지 좌표로 변환
        x1 = int((x_center - width / 2) * w)
        y1 = int((y_center - height / 2) * h)
        x2 = int((x_center + width / 2) * w)
        y2 = int((y_center + height / 2) * h)

        # 박스와 텍스트 추가
        color = (255, 0, 0)  # 빨간색 박스
        label = f"{class_names[class_id]}: {score:.2f}"
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    plt.imshow(image)
    plt.show()

# 클래스별, 점수별로 시각화
for class_id in class_names:
    for score_min in [0.0, 0.2, 0.4, 0.6, 0.8]:
        score_max = score_min + 0.2
        print(f"Class: {class_names[class_id]}, Score Range: {score_min} - {score_max}")

        # 예측 파일 반복
        for prediction_file in os.listdir(prediction_root):
            if prediction_file.endswith('.txt'):
                image_name = prediction_file.replace('.txt', '.jpg')
                image_path = os.path.join(data_root, image_name)
                prediction_path = os.path.join(prediction_root, prediction_file)
                
                predictions = read_prediction_file(prediction_path)
                visualize_predictions(image_path, predictions, class_filter=class_id, score_min=score_min, score_max=score_max)